In [1]:
import json
import pickle
import tqdm
import pandas as pd
from conversation import create_coder, create_reviewer, create_refiner, start_conversation

# Trabalho de RL - Qual o Melhor Prompt para Iterar sobre a Geração de Código?
Nosso trabalho de RL é sobre descobrir quais os melhores prompts para iterar sobre a geração de 
código de LLMs.

## Como funciona
Criamos uma conversa com 3 participantes: Coder, Reviewer e Refiner.
Cada participante é responsável por enviar um prompt para o LLM, e inputar os resultados do LLM no
ambiente (aqui chamado de 'conversa').

### Coder
O Coder é responsável por escrever o código.  
Ele envia o prompt inicial, descrevendo o problema a ser resolvido. Definimos que todos os nossos
problemas serão de limpeza de uma base de dados csv.  
O Coder só participa da conversa 1 vez (no início) e, por isso, não o definimos como um agente RL.
Ao invés disso, ele é programado para iterar por todos os prompts n vezes, e avaliamos os resultados
das conversas com cada prompt inicial posteriormente.

### Reviewer
O Reviewer é responsável por avaliar o código gerado pelo LLM.  
Ele envia um prompt solicitando a avaliação do código gerado pelo LLM. Ele pode essa avaliação
sempre após a geração de um código que não tem nota superior à nota terminal.  
O Reviewer é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Refiner
O Refiner é responsável por refinar o código gerado pelo LLM.
Ele envia um prompt solicitando a melhoria do código gerado pelo LLM. Ele pode essa avaliação
sempre após uma revisão do Reviewer.  
O Refiner é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Prompt
Para cada participante, geramos prompts que iam de 1 a $n$ nas **escalas** das seguintes **propriedades**:
- Clareza;
- Comprimento;
- Especificidade, e
- Complexidade.

Isso totalizou até 20 prompts diferentes para cada participante. Para diminuir o espaço de ações,
optamos por usar uma estratégia mais simples:

- Para cada prompt (**comprimento da escala** x **número de propriedades**) do **Coder**;
    - Para cada **propriedade** do **Reviewer**;
        - Para cada **propriedade** do **Refiner**;
            - Geramos $m$ conversas onde:
                1. O Coder envia o prompt e adiciona o código inicial;
                2. O código é avaliado (se a nota não for terminal, prossegue);
                3. O Reviewer escolhe um dos $n$ prompts da **propriedade** e adiciona a revisão;
                4. O Refiner escolhe um dos $n$ prompts da **propriedade** e adiciona a melhoria.
                5. Se o comprimento da conversa não for terminal, volta ao passo 2.

### Avaliação do Código
O código é avaliado por um LLM usando a bibliteca `instructor`. Pedimos que o código receba uma nota
de 0 a 100 para a sua corretude e legibilidade, bem como uma curta explicação do porquê da nota 
(esse comentário é adicionado posteriormente à conversa).  
Se a nota média for superior a 95, a conversa é terminada pois consideramos que o código é bom o
suficiente.

In [2]:
# List with JSON files name
json_files_coder = [
    "json_files/prompts_clarity_coder.json",
    "json_files/prompts_size_coder.json",
    "json_files/prompts_specificity_coder.json",
    "json_files/prompts_complexity_coder.json"
]

json_files_reviewer = [
    "json_files/prompts_clarity_reviewer.json",
    "json_files/prompts_size_reviewer.json",
    "json_files/prompts_specificity_reviewer.json",
    "json_files/prompts_complexity_reviewer.json"
]

json_files_refiner = [
    "json_files/prompts_prop1_refiner.json",
    "json_files/prompts_prop2_refiner.json",
    "json_files/prompts_prop3_refiner.json",
    "json_files/prompts_prop4_refiner.json"
]

prompts_coder = []
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        prompts_coder += data

reviewer_properties = {}    
for file_name in json_files_reviewer:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in reviewer_properties:
                reviewer_properties[item["propriedade"]] = []
            reviewer_properties[item["propriedade"]].append(item['prompt'])

refiner_properties = {}
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in refiner_properties:
                refiner_properties[item["propriedade"]] = []
            refiner_properties[item["propriedade"]].append(item['prompt'])          

In [3]:
coder_prompts = []
for coder_prompt_dict in prompts_coder:
    coder_prompts.append(coder_prompt_dict["prompt"])

csv_data = pd.read_csv("csv_tools/data.csv").to_string()

# Adding csv in coder prompts
for coder_prompt in coder_prompts:
    coder_prompt += f"\n\n Here is the CSV content:\n{csv_data}"  
    print(coder_prompt)

I gotta clean up this messed-up CSV file...  Python, please! 🥺

 Here is the CSV content:
                                         Series_Title  Released_Year  Runtime                         Genre  IMDB_Rating  Meta_score              Director  No_of_Votes
0                            The Shawshank Redemption            NaN  142 min                           NaN          NaN         NaN                   NaN          NaN
1                                                 NaN            NaN  175 min                           NaN          NaN         NaN  Francis Ford Coppola          NaN
2                                     The Dark Knight            NaN  152 min          Action, Crime, Drama          NaN         NaN     Christopher Nolan          NaN
3                                                 NaN         1974.0  202 min                  Crime, Drama          9.0         NaN  Francis Ford Coppola    1129952.0
4                                        12 Angry Men         1957.0  

In [ ]:
'''
MAX_TURNS = 1
TOT_CONVERSATIONS = 2
coder = create_coder(prompts_coder)

for coder_prompt_dict in prompts_coder:
    for rev_prop, rev_prompts in reviewer_properties.items():
        for ref_prop, ref_prompts in refiner_properties.items():
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                try:
                    start_conversation(
                        coder, 
                        coder_prompt_dict["prompt], 
                        reviewer, 
                        refiner, 
                        MAX_TURNS
                    )
                except Exception as e:
                    print(f"Conversation Skipped: {e}")
            # Saving the models
            with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(reviewer, file)
            with open(f"models/refiner_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(refiner, file)
'''                

Conversations:  50%|█████     | 1/2 [01:13<01:13, 73.96s/it]


KeyboardInterrupt: 

In [ ]:
from IPython.display import display, Markdown
import os
import re

# Ensuring that file names are valid
def sanitize_filename(filename):
    return re.sub(r'[^A-Za-z0-9_]', '_', filename)

# Ensuring the 'models' directory exists
os.makedirs("models", exist_ok=True)

MAX_TURNS = 1
TOT_CONVERSATIONS = 2
coder = create_coder(prompts_coder)

for coder_prompt in coder_prompts:
    sanitized_coder_prompt = sanitize_filename(coder_prompt)
    for rev_prop, rev_prompts in reviewer_properties.items():
        sanitized_rev_prop = sanitize_filename(rev_prop)
        for ref_prop, ref_prompts in refiner_properties.items():
            sanitized_ref_prop = sanitize_filename(ref_prop)
            
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                try:
                    environment = start_conversation(
                        coder, 
                        coder_prompt, 
                        reviewer, 
                        refiner, 
                        MAX_TURNS
                    )
                    for message in environment.messages:
                        display(Markdown(f"**{message['role']}**: {message['content']}"))
                except Exception as e:
                    print(f"Conversation Skipped: {e}")
            # Saving the models
            with open(f"models/reviewer_{sanitized_coder_prompt}_{sanitized_rev_prop}_{sanitized_ref_prop}.pkl", "wb") as file:
                pickle.dump(reviewer, file)
            with open(f"models/refiner_{sanitized_coder_prompt}_{sanitized_rev_prop}_{sanitized_ref_prop}.pkl", "wb") as file:
                pickle.dump(refiner, file)

Conversations: 100%|██████████| 2/2 [00:00<?, ?it/s]


Conversation Skipped: string indices must be integers, not 'str'
Conversation Skipped: string indices must be integers, not 'str'


Conversations: 100%|██████████| 2/2 [00:00<?, ?it/s]

Conversation Skipped: string indices must be integers, not 'str'
Conversation Skipped: string indices must be integers, not 'str'


Conversations: 100%|██████████| 2/2 [00:00<00:00, 768.05it/s]


...

Conversations: 100%|██████████| 2/2 [00:00<?, ?it/s]


Conversation Skipped: string indices must be integers, not 'str'
Conversation Skipped: string indices must be integers, not 'str'


Conversations: 100%|██████████| 2/2 [00:00<?, ?it/s]


Conversation Skipped: string indices must be integers, not 'str'
Conversation Skipped: string indices must be integers, not 'str'


FileNotFoundError: [Errno 2] No such file or directory: 'models/reviewer_I_need_help_cleaning_up_this_messy_CSV_file_using_Python__Can_you_write_a_script_that_can_handle_the_cleaning_and_data_manipulation_tasks_efficiently__clarity_specificity.pkl'